In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

# Web sitesine başlık bilgisi ekleyerek requests modülü aracılığıyla bağlanıyoruz.
headers_param = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
url = "https://www.accuweather.com/tr/tr/ankara/316938/daily-weather-forecast/316938"
r = requests.get(url, headers=headers_param)

# İsteğe bağlı içerik alınıyor ve BeautifulSoup kullanılarak ağaç yapısı oluşturuluyor.
soup = BeautifulSoup(r.content, "lxml")

# Hedeflenen verinin içeriğini bulmak için gerekli HTML etiketlerini buluyoruz.
date_container = soup.find("div", attrs={"class": "two-column-page-content"})
daily = date_container.find_all("div", attrs={"class": "daily-wrapper"})

# Veriyi saklamak için boş listeler oluşturuluyor.
days_of_week = []
dates = []
max_temperatures = []
min_temperatures = []
weather_conditions = []

# Her bir günlük öğe için döngü oluşturuluyor ve veri çekiliyor.
for day in daily:
    date_elem = day.find("h2", attrs={"class": "date"})
    
    # Gün ve tarih bilgisi çekiliyor ve uygun listelere ekleniyor.
    days_of_week.append(date_elem.find("span", attrs={"class": "module-header dow date"}).text)
    dates.append(date_elem.find("span", attrs={"class": "module-header sub date"}).text)
    
    # Maksimum ve minimum sıcaklık bilgisi çekiliyor ve uygun listelere ekleniyor.
    max_temp = day.find("div", attrs={"class": "temp"}).select("span:nth-of-type(1)")
    max_temperatures.append(max_temp[0].string)
    min_temp = day.find("div", attrs={"class": "temp"}).select("span:nth-of-type(2)")
    min_temperatures.append(min_temp[0].string.replace("/", ""))
    
    # Hava durumu koşulu bilgisi çekiliyor ve uygun listeye ekleniyor.
    condition = day.find("div", attrs={"class": "phrase"}).string.replace("\t", "")
    weather_conditions.append(condition.replace("\n", ""))

# Toplanan veriler bir sözlükte saklanıyor.
weather_data = {'Date': dates, 'Day': days_of_week, 'Max Temperature': max_temperatures, 'Min Temperature': min_temperatures, 'Condition': weather_conditions}

# Verileri bir DataFrame'e dönüştürüyoruz.
df = pd.DataFrame(weather_data)

# CSV dosyasına veri yazmak için dosyayı açıyoruz ve csv.DictWriter kullanarak başlık ve satırları yazıyoruz.
with open('C:\\Users\\fatih\\Documents\\havaDurumu.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=weather_data.keys())
    writer.writeheader()
    for row in df.to_dict(orient='records'):
        writer.writerow(row)
